In [14]:
import pandas as pd
import nltk
nltk.download('stopwords')
import re

import spacy
nlp = spacy.load('en_core_web_sm')

from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer

import warnings
warnings.filterwarnings("ignore", category=FutureWarning) 
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/michelle/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


use cases: we want to have some automatic custom tagging of job descriptions
to do this, we can mine some keywords and group similar jobs together

In [2]:
df = pd.read_json("ww_data_1.json")
df.head()

,280880,280881,280886,280889,280890,280891,280892,282671,284396,285124,...,292109,292110,292112,292202,292219,292221,292251,292409,292454,292498
Application Information,{'Additional Application Information': '<stron...,{'Additional Application Information': '<stron...,{'Additional Application Information': '<stron...,{'Additional Application Information': '<stron...,{'Additional Application Information': '<stron...,{'Additional Application Information': '<stron...,{'Additional Application Information': '<stron...,{'Application Deadline': 'October ...,{'Additional Application Information': '<stron...,{'Additional Application Information': 'DTRC's...,...,{'Additional Application Information': 'This i...,{'Additional Application Information': 'We tha...,{'Additional Application Information': 'Live i...,{'Additional Application Information': 'We are...,{'Additional Application Information': 'Intere...,{'Application Deadline': 'October ...,{'Additional Application Information': '?Shoul...,{'Additional Application Information': 'Excite...,{'Application Deadline': 'October ...,{'Additional Application Information': '<stron...
Company Information,"{'Division': 'Assurance', 'Organization': 'PwC'}","{'Division': 'Assurance', 'Organization': 'PwC'}","{'Division': 'Assurance', 'Organization': 'PwC'}","{'Division': 'Assurance', 'Organization': 'PwC'}","{'Division': 'Assurance', 'Organization': 'PwC'}","{'Division': 'Assurance', 'Organization': 'PwC'}","{'Division': 'Assurance', 'Organization': 'PwC'}","{'Division': 'IKO Canada 2', 'Organization': '...","{'Division': 'Divisional Office', 'Organizatio...","{'Division': 'Divisional Office', 'Organizatio...",...,"{'Division': 'L'Arche Ontario', 'Organization'...","{'Division': 'Head Office', 'Organization': 'T...","{'Division': 'L'Arche Ontario', 'Organization'...","{'Division': 'Head Office', 'Organization': 'B...","{'Division': 'Ottawa Office', 'Organization': ...","{'Division': 'Divisional Office', 'Organizatio...","{'Division': 'Divisional Office', 'Organizatio...","{'Division': 'Divisional Office', 'Organizatio...","{'Division': 'Divisional Office', 'Organizatio...","{'Division': 'Head Office', 'Organization': 'Z..."
Job Posting Information,{'Additional Job Identifiers': '<table> ...,{'Additional Job Identifiers': '<table> ...,{'Additional Job Identifiers': '<table> ...,{'Additional Job Identifiers': '<table> ...,{'Additional Job Identifiers': '<table> ...,{'Additional Job Identifiers': '<table> ...,{'Additional Job Identifiers': '<table> ...,{'Job - Address Line One': '1900 Minnesota Cou...,{'': '<em>Canada's Top 100 Employers<br> (2...,{'Compensation and Benefits Information': '<st...,...,{'Additional Information': '<table> ...,{'Compensation and Benefits Information': 'We ...,{'Additional Information': '<table> ...,{'Additional Information': '<table> ...,{'Compensation and Benefits Information': 'Wha...,{'Additional Information': '<table> ...,{'Additional Information': '<table> ...,{'Additional Information': '<table> ...,{'Compensation and Benefits Information': '$35...,{'Job - Address Line One': '250 The Esplanade'...
Posting List Data,"{'applications': '11', 'company': 'PwC', 'dead...","{'applications': '7', 'company': 'PwC', 'deadl...","{'applications': '6', 'company': 'PwC', 'deadl...","{'applications': '9', 'company': 'PwC', 'deadl...","{'applications': '2', 'company': 'PwC', 'deadl...","{'applications': '2', 'company': 'PwC', 'deadl...","{'applications': '9', 'company': 'PwC', 'deadl...","{'applications': '21', 'company': 'IKO Industr...","{'applications': '14', 'company': 'Mattamy Hom...","{'applications': '18', 'company': 'Dancer Tran...",...,"{'applications': '0', 'company': 'L'Arche Cana...","{'applications': '47', 'company': 'TradeRev', ...","{'applications': '0', 'company': 'L'Arche Cana...","{'applications': '0', 'company': 'Boszko & Ver...","{'applications': '15', 'company': 'Nokia', 'de...","{'applications': '3', 'company': 'Innovation S...","{'applications': '7', 'compa

In [3]:
df_final = pd.DataFrame(columns = ['id','job_title','job_summary','job_responsibilities','required_skills','duration'])

for x in df.columns:
    id = x
    title = df[x][3]['jobTitle']
    summary = ""
    try: 
        summary = df[x][2]['Job Summary']
    except:
        summary = ""
    responsibilities = df[x][2]['Job Responsibilities']
    requirements = df[x][2]['Required Skills']
    
    df_final = df_final.append({'id':id,'job_title':title,'job_summary':summary,'job_responsibilities':responsibilities,'required_skills':requirements},ignore_index=True)
    df_final = df_final.drop_duplicates()

In [4]:
df_final

,id,job_title,job_summary,job_responsibilities,required_skills,duration
0,280880,January 2023 Tax MySolutions- 4 Month Co-op (C...,<strong>Interested applicants must apply throu...,<strong>What you'll create and do</strong> \n<...,<strong>What you'll bring to this role:</stron...,NaN
1,280881,January 2023 Tax MySolutions- 4 Month Co-op (C...,<strong>Interested applicants must apply throu...,<strong>What you'll create and do</strong> \n<...,<strong>What you'll bring to this role:</stron...,NaN
2,280886,January 2023 Tax MySolutions- 8 Month Co-op (C...,<strong>Interested applicants must apply throu...,<strong>What you'll create and do</strong> \n<...,<strong>What you'll bring to this role:</stron...,NaN
3,280889,January 2023 Tax MySolutions- 8 Month Co-op (C...,<strong>Interested applicants must apply throu...,<strong>What you'll create and do</strong> \n<...,<strong>What you'll bring to this role:</stron...,NaN
4,280890,January 2023 Tax MySolutions- 8 Month Co-op (C...,<strong>Interested applicants must apply throu...,<strong>What you'll create and do</strong> \n<...,<strong>What you'll bring to this role:</stron...,NaN
...,...,...,...,...,...,...
423,292221,Junior Policy Analyst,"Are you interested in technology, cutting-edge...","<a name=""_Hlk92288386""></a> \n<br>\nDuties may...","<a name=""_Hlk92288398""><strong>Essential skill...",NaN
424,292251,Junior Accountant,WinnerMax Capital Inc. is a Canadian instituti...,What will you do \n<br>\n&nbsp; \n<br>\nWorkin...,What you need to succeed \n<br>\n&nbsp; \n<br>...,NaN
425,292409,Experiential Marketing + Event & Sampling Coor...,<strong>Who we are:&nbsp;</strong> \n<br>\nAt ...,<strong>Experiential Marketing + Event &amp; S...,<strong>You have:&nbsp;</strong>\n\n \n<ul>\n ...,NaN
426,292454,"Software Engineering: Kernel, Systems, Memory,...",MemVerge is seeking software developers who de...,A Few of Your Key Responsibilities could inclu...,<strong>Software Engineer - Systems Software</...,NaN


In [12]:
doc = df_final['job_responsibilities'][0]
doc

"<strong>What you'll create and do</strong> \n<br>\nPwC's Tax professionals understand how businesses work from the inside. Our Private Company Services 'MySolutions' group practices across Canada and works closely with private companies to help solve business issues faced by shareholders and management using a multi-disciplinary approach. From assurance, financial and compliance reporting to tax strategies and returns, our team works to deliver long-term results that balance business and shareholder goals.&nbsp;&nbsp; \n<br>\nOur teams ask questions, review assumptions and evaluate whether companies are reporting information upon which stakeholders can rely on. We combine accounting, financial reporting and tax skills with business knowledge and experience to help our clients address the challenges of a rapidly changing environment and strengthen their businesses. We're true business advisors. \n<br>\nAs an Associate working towards your CPA designation you'll be put at the heart of u

In [16]:
def no_number_preprocessor(tokens):
    r = re.sub('(\d)+', 'NUM', tokens.lower())
    # This alternative just removes numbers:
    # r = re.sub('(\d)+', '', tokens.lower())
    return r

In [17]:

sw_custom = stopwords.words('english')
sw_custom.append("2021")
sw_custom.append("2022")
sw_custom.append("2017")
sw_custom.append("students")
sw_custom.append("student")
sw_custom.append("january")
print(sw_custom)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [18]:

n_gram_range = (1, 1)
stop_words = "english"

# Extract candidate words/phrases
count = CountVectorizer(ngram_range=n_gram_range, stop_words=sw_custom,preprocessor = no_number_preprocessor).fit([doc])
candidates = count.get_feature_names()



/Users/michelle/miniconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['NUM'] not in stop_words.
  warnings.warn(


In [19]:
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
doc_embedding = model.encode([doc])
candidate_embeddings = model.encode(candidates)

In [26]:
top_n = 7
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]

In [27]:
keywords

['business',
 'shareholders',
 'companies',
 'colleagues',
 'businesses',
 'accounting',
 'canada']

## new iteration of model testing

df = pd.read_json("ww_data_2920_jobs_w_wtr.json")

In [ ]:
df = df.drop("division")
df = df.drop("graphs")
df = df.drop("hire_history")
df = df.drop("organization")
df = df.dropna(axis=1)

### extract relevant info from data

In [ ]:
df_final = pd.DataFrame(columns = ['id','job_title','job_summary','job_responsibilities','required_skills','duration','special_job_reqs','app_docs'])

for x in df.columns:
    id = x
    try:
        title = df[x][3]['Job Title']
    except:
        title = ""
    try: 
        summary = df[x][3]['Job Summary']
    except:
        summary = ""
    try:
        responsibilities = df[x][3]['Job Responsibilities']
    except:
        responsibilities = ""
    try:
        requirements = df[x][3]['Required Skills']
    except:
        requirements = ""
    try:
        duration = df[x][3]['Work Term Duration']
    except:
        duration = ""
    try:
        special_job_requirements = df[x][3]['Special Job Requirements']
    except:
        special_job_requirements = ""
    app_docs = df[x][0]['Application Documents Required']
    
    df_final = df_final.append({'id':id,'job_title':title,'job_summary':summary,
    'job_responsibilities':responsibilities,'required_skills':requirements,
    'duration':duration, 'special_job_reqs':special_job_requirements,'app_docs':app_docs},ignore_index=True)
    df_final = df_final.drop_duplicates()
    df_final['text'] = df_final['job_responsibilities'] + df_final['required_skills']

In [ ]:
def get_reqs(text):
    reqs = []
    if (('University of Waterloo Co-op Work History') in text):
        reqs.append('work history')
    if('Cover Letter' in text):
        reqs.append('cover letter')
    if('Résumé' in text):
        reqs.append('resume')
    if('Grade Report' in text):
        reqs.append("grade report")
    if('Other' in text):
        reqs.append('other - per job posting')
    return reqs

def get_duration(text):
    if ('4 month' in text):
        return '4 month'
    if ('8 month' in text):
        if('preferred' in text):
            return '8 month preferred'
        if('required' in text):
            return '8 month required'
    if ('2 work term' in text):
        if('preferred' in text):
            return '2 work terms preferred'
        if('required' in text):
            return '2 work terms required'
    return text

def get_special_reqs(text):
    list = []
    if('directly to the employer' in text):
        list.append("external application")
    if ('Canadian citizen' in text):
        list.append("Canadian citizenship")
    if('permanent resident' in text):
        list.append("permanent resident")
    if('fully vaccinated' in text):
        list.append("fully vaccinated")
    if('eligible to work in the USA' in text or 'USA visa' in text):
        list.append("USA work eligibility")
    if('This job requires you to work remotely from CANADA' in text):
        list.append("remote requirement Canada")
    if('Security Clearance' in text):
        list.append('security clearance')
    
    return list

In [ ]:
df_final['app_docs_tags'] = df_final.apply(lambda row: get_reqs(row['app_docs']),axis=1)
df_final['duration_tags'] = df_final.apply(lambda row: get_duration(row['duration']),axis=1)
df_final['special_reqs_tags'] = df_final.apply(lambda row: get_special_reqs(row['special_job_reqs']),axis=1)

In [ ]:
clean_html = re.compile('<.*?>') 
def sanitize(text):
    #normalize and remove html
    text = re.sub(clean_html, '', text)
    text = text.lower()
    text = re.sub("&nbsp;","",text)
    text = re.sub("&amp;","",text)
    text = re.sub("\\n","",text)
    text = re.sub("\\'","\'",text)
    text = re.sub("\\t","",text)
    text=re.sub("    ","",text)
    #text = re.sub(" ?","",text)
    #remove unicode
    text = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", text)
    #remove misc
    text = re.sub(" job description  summary ","",text)
    return text

In [ ]:
df_final['sanitized'] = df_final.apply(lambda row: sanitize(row['text']),axis=1)

In [ ]:
def no_number_preprocessor(tokens):
    r = re.sub('(\d)+', 'NUM', tokens.lower())
    # This alternative just removes numbers:
    # r = re.sub('(\d)+', '', tokens.lower())
    return r

### create custom stopwords

In [ ]:
sw_custom = ['coop','intern''job','working','workplace','abovesee','lunch','university','undergraduate','studies','deadlines','exceptional','working','hours','classes','wellness','national','pride','interns','new','outstanding','april',
'customers','drivers','work','office',
'workday','month','internship','job','seeabove','co','op','co-op','li',
'clients', 'examples', 'questions', 'experience', 'postsecondary', 'position', 'group','services',
'priorities', 'requirement', 'specialists', 'others', 'member','role', 'preference', 'commitment', 'ability', 
'stakeholders','skills', 'people', 'challenges', 'opportunity', 'interest','company','waterlooworks','companies',
'colleagues','applicants','students','interested applicants','interested','student','january','companies','company','advisors']
stop_words = stopwords.words('english')
stop_words.extend(sw_custom)

## first keyword model

In [ ]:
n_gram_range = (1, 2)
stop_words = stopwords.words('english')
stop_words.extend(sw_custom)

model = SentenceTransformer('distilbert-base-nli-mean-tokens')

counter = 0

def get_keywords(text,n):
    try:
        count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words,preprocessor = no_number_preprocessor).fit([text])
        all_candidates = count.get_feature_names()
    except:
        return []
    doc = nlp(text)
    noun_phrases = set(chunk.text.strip().lower() for chunk in doc.noun_chunks)

    nouns = set()
    for token in doc:
        if token.pos_ == "NOUN":
            nouns.add(token.text)
    all_nouns = nouns.union(noun_phrases)
    candidates = list(filter(lambda candidate: candidate in all_nouns, all_candidates))

    doc_embedding = model.encode([text])
    candidate_embeddings = model.encode(candidates)

    top_n = n
    try:
        distances = cosine_similarity(doc_embedding, candidate_embeddings)
    except:
        return []
    keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
    return keywords


In [ ]:
df_final['keywords'] = df_final.apply(lambda row: get_keywords(row['text'],5),axis=1)

## second keyword model

In [ ]:
from transformers import AutoModel, AutoTokenizer
model_name = "distilroberta-base"
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, model_max_length=512)

def get_keywords_2(text,n):
    count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words,preprocessor = no_number_preprocessor).fit([text])
    all_candidates = count.get_feature_names()
    doc = nlp(text)
    noun_phrases = set(chunk.text.strip().lower() for chunk in doc.noun_chunks)

    nouns = set()
    for token in doc:
        if token.pos_ == "NOUN":
            nouns.add(token.text)
    all_nouns = nouns.union(noun_phrases)
    candidates = list(filter(lambda candidate: candidate in all_nouns, all_candidates))
    candidate_tokens = tokenizer(candidates, padding=True, truncation=True, max_length = 512,return_tensors="pt")
    candidate_embeddings = model(**candidate_tokens)["pooler_output"]
    text_tokens = tokenizer([text], padding=True, truncation=True, max_length = 512,return_tensors="pt")
    text_embedding = model(**text_tokens)["pooler_output"]

    candidate_embeddings = candidate_embeddings.detach().numpy()
    text_embedding = text_embedding.detach().numpy()

    top_n = n
    distances = cosine_similarity(text_embedding, candidate_embeddings)
    keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
    return keywords

In [ ]:
df_final['keywords_2'] = df_final.apply(lambda row: get_keywords_2(row['text'],5),axis=1)

## try pre-trained model keybert

In [ ]:
from keybert import KeyBERT
kw_model = KeyBERT()

In [ ]:
def extract_keybert(text,top_n):
    keywords = kw_model.extract_keywords(text,stop_words = stop_words,top_n=top_n)
    return keywords

In [ ]:
sw_custom = ['requirements','coop','intern','duties','initiative','assistive','qualifications','job','working','workplace','abovesee','lunch','university','undergraduate','studies','deadlines','exceptional','working','hours','classes','wellness','national','pride','interns','new','outstanding','april',
'customers','drivers','work','office', 'responsibilities', 'projects', 'documents',
'workday','month','internship','job','seeabove','co','op','co-op','li',
'clients', 'examples', 'questions', 'experience', 'postsecondary', 'position', 'group','services',
'priorities', 'requirement', 'specialists', 'others', 'member','role', 'preference', 'commitment', 'ability', 
'stakeholders','skills', 'people', 'challenges', 'opportunity', 'interest','company','waterlooworks','companies',
'colleagues','applicants','students','interested applicants','interested','student','january','companies','company','advisors']
stop_words = stopwords.words('english')
stop_words.extend(sw_custom)

In [ ]:
df_final['keywords'] = df_final.apply(lambda row: extract_keybert(row['sanitized'],5),axis=1)